In [4]:
from model.ResNet34_attention import ResidualAttentionModel_92_32input_update as ResidualAttentionModel
import torch

model = ResidualAttentionModel()
sample_input = torch.randn(1, 3, 32, 32)

In [6]:
model

ResidualAttentionModel_92_32input_update(
  (conv1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (residual_block1): ResidualBlock(
    (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (conv1): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3): Conv2d(32, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (conv4): Conv2d(32, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
  )
  (attention_module1): AttentionModule_stage1_cifar(
    (first_re

In [5]:
model(sample_input)

tensor([[ 0.1550,  0.0206, -0.4204,  0.2909, -0.2714,  0.2896, -0.0239, -0.2595,
         -0.1206, -0.1068]], grad_fn=<AddmmBackward0>)